# Adapter (no caching)

> Making interfaces talk to each other, inefficiently.

Let's imagine we are given a prepackaged API that consists of a `Point` class as well as a `draw_point` function:

In [ ]:
class Point:
    def __init__(self, x, y):
        self.y = y
        self.x = x

def draw_point(p):
    print('.', end='') # we will assume this actually prints the Point, and we will avoid doing line breaks after printing in order to see multiple dots in the same line.

So we are given this API and we simply must work with it and there's nothing we can do, but it so happens that in our application, everything is made of lines, which we use to make larger objects like rectangles, like this:

In [ ]:
class Line:
    def __init__(self, start, end):
        self.end = end
        self.start = start
        
class Rectangle(list):
    """ Represented as a list of lines. """
    def __init__(self, x, y, width, height):
        super().__init__()
        self.append(Line(Point(x, y), Point(x + width, y)))
        self.append(Line(Point(x + width, y), Point(x + width, y + height)))
        self.append(Line(Point(x, y), Point(x, y + height)))
        self.append(Line(Point(x, y + height), Point(x + width, y + height)))

If we wanted to create some rectangles, it would look something like this:

In [3]:
rs = [
    Rectangle(1, 1, 10, 10),
    Rectangle(3, 3, 6, 6)
]

But we have a problem: the API we're given allows us to draw points but our rectangles are line based, and even though lines have a start and end point, in order for the drawing function to work we need every single point that makes the line, which we simply don't have.

So, how do we use the given API with our object? We build an **adapter**: an in-between component that will make this possible.

In our case, what we need is to represent a line as a series of points in order to make use of the `draw_point` function. We will create a new class that converts a line into points:

In [4]:
class LineToPointAdapter(list):
    """ Represented as a list of points"""
    
    count = 0 # to keep track of how many points we've generated

    def __init__(self, line): # we specify the line we're adapting
        self.count += 1
        print(f'{self.count}: Generating points for line '
              f'[{line.start.x},{line.start.y}]→'
              f'[{line.end.x},{line.end.y}]')

        # we calculate the coordinates of the margins of the line
        left = min(line.start.x, line.end.x)
        right = max(line.start.x, line.end.x)
        top = min(line.start.y, line.end.y)
        bottom = min(line.start.y, line.end.y)

        # Finally, we incrementally create the points along the coordinates and we append them to our list (the LineToPointAdapter instance)
        if right - left == 0:
            for y in range(top, bottom):
                self.append(Point(left, y))
        elif line.end.y - line.start.y == 0:
            for x in range(left, right):
                self.append(Point(x, top))
        

With our adapter in place, we can create a new drawing method that adapts our lines to points and prints them with the point drawing API:

In [5]:
def draw(rcs):
    print("\n\n--- Drawing some stuff ---\n")
    for rc in rcs:
        for line in rc:
            adapter = LineToPointAdapter(line)
            for p in adapter:
                draw_point(p)

Let's see it in action:

In [6]:
rs = [
    Rectangle(1, 1, 10, 10),
    Rectangle(3, 3, 6, 6)
]
draw(rs)



--- Drawing some stuff ---

1: Generating points for line [1,1]→[11,1]
..........1: Generating points for line [11,1]→[11,11]
1: Generating points for line [1,1]→[1,11]
1: Generating points for line [1,11]→[11,11]
..........1: Generating points for line [3,3]→[9,3]
......1: Generating points for line [9,3]→[9,9]
1: Generating points for line [3,3]→[3,9]
1: Generating points for line [3,9]→[9,9]
......

This works, but we have a small problem: if we duplicate the  `draw` call, we will generate the points twice:

In [7]:
rs = [
    Rectangle(1, 1, 10, 10),
    Rectangle(3, 3, 6, 6)
]
draw(rs)
draw(rs)



--- Drawing some stuff ---

1: Generating points for line [1,1]→[11,1]
..........1: Generating points for line [11,1]→[11,11]
1: Generating points for line [1,1]→[1,11]
1: Generating points for line [1,11]→[11,11]
..........1: Generating points for line [3,3]→[9,3]
......1: Generating points for line [9,3]→[9,9]
1: Generating points for line [3,3]→[3,9]
1: Generating points for line [3,9]→[9,9]
......

--- Drawing some stuff ---

1: Generating points for line [1,1]→[11,1]
..........1: Generating points for line [11,1]→[11,11]
1: Generating points for line [1,1]→[1,11]
1: Generating points for line [1,11]→[11,11]
..........1: Generating points for line [3,3]→[9,3]
......1: Generating points for line [9,3]→[9,9]
1: Generating points for line [3,3]→[3,9]
1: Generating points for line [3,9]→[9,9]
......

Out adapter is generating temporary objects which are discarded, and while this doesn't matter that much in this example, in other use cases with complex adapters this can be very costly.

In the next lesson we will see how to optimize this problem.